In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext yamlmagic

In [ ]:
"""
Generate some fake data with noise in it just in case

"""

from sklearn.datasets import fetch_olivetti_faces
from skimage.util import random_noise

images = fetch_olivetti_faces()["images"]

# Add some noise
noisy_images = random_noise(images, mode="speckle", var=0.01)

In [ ]:
"""
Plot an example

"""

import numpy as np
import matplotlib.pyplot as plt

# Plot both
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot clean data
plot_kw = {"cmap": "grey", "aspect": "auto", "vmin": 0, "vmax": 1}
im1 = axes[0].imshow(images[0], **plot_kw)
im2 = axes[1].imshow(noisy_images[0], **plot_kw)
im3 = axes[2].imshow(noisy_images[0] - images[0], **plot_kw)

for axis, im, label in zip(axes, [im1, im2, im3], ["Clean", "Noisy", "Difference"]):
    axis.set_title(label)
    axis.set_xlabel("X")
    axis.set_ylabel("Y")
    plt.colorbar(im, ax=axis)

fig.tight_layout()

In [ ]:
%%yaml config

n_epochs: 100
batch_size: 64
learning_rate: 0.0002
n_cpu: 8
latent_dim: 100
img_shape: [64, 64]
channels: 1

In [ ]:
"""
Define the GAN

"""
from current_denoising.generation import dcgan

In [ ]:
"""
Train the GAN

"""

In [ ]:
"""
Show some generated patches

"""